In [1]:
import sqlite3
import os

# Nama file database
DB_FILE = 'hospital_db.sqlite'
# Nama file kueri
QUERY_FILE = 'test_data_logic.sql'

def create_database():
    """
    Membuat tabel-tabel (patients, visits, symptoms) 
    sesuai skema di file PDF.
    """
    # Hapus file database lama jika ada
    if os.path.exists(DB_FILE):
        os.remove(DB_FILE)
        print(f"Menghapus database lama: {DB_FILE}")

    # Hubungkan ke database (ini akan membuat file baru)
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # Aktifkan foreign key support
    cursor.execute("PRAGMA foreign_keys = ON;")

    print("Membuat tabel...")
    
    # Skema Tabel 1: patients
    cursor.execute("""
    CREATE TABLE patients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER NOT NULL
    );
    """)

    # Skema Tabel 2: visits
    cursor.execute("""
    CREATE TABLE visits (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id INTEGER NOT NULL,
        department TEXT NOT NULL,
        visit_date DATE NOT NULL,
        FOREIGN KEY (patient_id) REFERENCES patients (id)
    );
    """)

    # Skema Tabel 3: symptoms
    cursor.execute("""
    CREATE TABLE symptoms (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        visit_id INTEGER NOT NULL,
        symptom TEXT NOT NULL,
        FOREIGN KEY (visit_id) REFERENCES visits (id)
    );
    """)

    print("Tabel berhasil dibuat.")
    conn.commit()
    conn.close()

def populate_data():
    """
    Mengisi database dengan data contoh yang relevan 
    dengan kueri (Kasus 2), menggunakan data pasien dari CSV.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    print("Mengisi data contoh (disesuaikan dengan patients.csv)...")

    # --- Data Pasien (diambil dari patients.csv) ---
    # Kriteria Kueri: Usia > 50
    # Pasien yang relevan: Citra (62), Idul (88), Jaka (54)
    patients_data = [
        (1, 'Andi', 45),
        (2, 'Budi', 29),
        (3, 'Citra', 62),    # (Target Usia)
        (4, 'Dita', 30),
        (5, 'Eka', 18),
        (6, 'Fitra', 49),
        (7, 'Gio', 49),
        (8, 'Harianto', 33),
        (9, 'Idul', 88),     # (Target Usia)
        (10, 'Jaka', 54)     # (Target Usia)
    ]
    cursor.executemany("INSERT INTO patients (id, name, age) VALUES (?, ?, ?)", patients_data)

    # --- Data Kunjungan ---
    # Kita buat skenario agar kueri (Neurology, >50 thn, >=3 gejala) 
    # menghasilkan data
    visits_data = [
        # (id, patient_id, department, visit_date)
        
        # --- TARGET (Akan muncul di hasil) ---
        (1, 3, 'Neurology', '2025-10-23'), # 1. Citra (62), 3 gejala, Terbaru
        (2, 9, 'Neurology', '2025-10-22'), # 2. Idul (88), 4 gejala
        (3, 10, 'Neurology', '2025-10-20'),# 3. Jaka (54), 3 gejala
        (4, 3, 'Neurology', '2025-10-15'), # 4. Citra (62), 3 gejala, lebih lama
        (5, 9, 'Neurology', '2025-10-01'), # 5. Idul (88), 3 gejala, terlama

        # --- NON-TARGET (Tidak akan muncul) ---
        (6, 1, 'Neurology', '2025-10-21'), # Andi (45), Usia < 50
        (7, 7, 'Neurology', '2025-10-19'), # Gio (49), Usia < 50
        (8, 3, 'Kardiologi', '2025-10-18'),# Citra (62), Dept != Neurology
        (9, 10, 'Neurology', '2025-10-17') # Jaka (54), Gejala < 3 (hanya 2)
    ]
    cursor.executemany("INSERT INTO visits (id, patient_id, department, visit_date) VALUES (?, ?, ?, ?)", visits_data)

    # --- Data Gejala ---
    # Disesuaikan dengan data kunjungan di atas
    symptoms_data = [
        # Visit 1 (Citra, Target)
        (None, 1, 'pusing'),
        (None, 1, 'kehilangan keseimbangan'),
        (None, 1, 'mual'),
        
        # Visit 2 (Idul, Target)
        (None, 2, 'susah tidur'),
        (None, 2, 'sakit kepala'),
        (None, 2, 'bingung'),
        (None, 2, 'lemas'),

        # Visit 3 (Jaka, Target)
        (None, 3, 'sesak nafas'),
        (None, 3, 'nyeri dada'),
        (None, 3, 'batuk'),

        # Visit 4 (Citra, Target)
        (None, 4, 'vertigo'),
        (None, 4, 'kesemutan'),
        (None, 4, 'sulit berjalan'),
        
        # Visit 5 (Idul, Target)
        (None, 5, 'sulit tidur'),
        (None, 5, 'cemas'),
        (None, 5, 'kaki kram'),

        # Visit 6 (Andi, Non-Target: Usia)
        (None, 6, 'demam'),
        (None, 6, 'batuk'),
        (None, 6, 'sesak napas'),

        # Visit 7 (Gio, Non-Target: Usia)
        (None, 7, 'menggigil'),
        (None, 7, 'batuk'),
        (None, 7, 'sakit kepala'),

        # Visit 8 (Citra, Non-Target: Dept)
        (None, 8, 'jantung berdebar'),
        (None, 8, 'pusing'),
        (None, 8, 'sesak'),

        # Visit 9 (Jaka, Non-Target: Gejala < 3)
        (None, 9, 'sesak nafas'),
        (None, 9, 'lemas')
    ]
    cursor.executemany("INSERT INTO symptoms (id, visit_id, symptom) VALUES (?, ?, ?)", symptoms_data)

    print("Data berhasil diisi.")
    conn.commit()
    conn.close()

def run_query():
    """
    Membaca kueri dari file .sql dan menjalankannya 
    di database yang telah dibuat.
    """
    if not os.path.exists(QUERY_FILE):
        print(f"Error: File kueri '{QUERY_FILE}' tidak ditemukan.")
        print("Pastikan Anda memiliki 'test_data_logic.sql' di folder yang sama.")
        return

    if not os.path.exists(DB_FILE):
        print(f"Error: File database '{DB_FILE}' tidak ditemukan.")
        print("Jalankan fungsi create_database() dan populate_data() terlebih dahulu.")
        return

    # Baca kueri dari file
    with open(QUERY_FILE, 'r', encoding='utf-8') as f:
        query = f.read()

    print(f"\nMenjalankan kueri dari {QUERY_FILE}...")

    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    results = cursor.execute(query).fetchall()

    print("--- HASIL KUERI (Top 5 Kunjungan Neurologi, >50 thn, >=3 gejala) ---")
    if not results:
        print("Tidak ada data yang cocok dengan kriteria.")
    else:
        # Cetak Header
        print(f"{'patients.name':<15} | {'patients.age':<12} | {'visits.visit_date':<17} | {'symptom_count':<15}")
        print("-" * 65)
        # Cetak Hasil
        for row in results:
            print(f"{row[0]:<15} | {row[1]:<12} | {row[2]:<17} | {row[3]:<15}")

    conn.close()


In [2]:
try:
    # 1. Buat struktur tabel
    create_database()
    
    # 2. Isi dengan data contoh
    populate_data()
    
    # 3. Jalankan kueri utama
    run_query()
except sqlite3.Error as e:
    print(f"Terjadi error SQLite: {e}")
except Exception as e:
    print(f"Terjadi error: {e}")



Membuat tabel...
Tabel berhasil dibuat.
Mengisi data contoh (disesuaikan dengan patients.csv)...
Data berhasil diisi.

Menjalankan kueri dari test_data_logic.sql...
--- HASIL KUERI (Top 5 Kunjungan Neurologi, >50 thn, >=3 gejala) ---
patients.name   | patients.age | visits.visit_date | symptom_count  
-----------------------------------------------------------------
Citra           | 62           | 2025-10-23        | 3              
Idul            | 88           | 2025-10-22        | 4              
Jaka            | 54           | 2025-10-20        | 3              
Citra           | 62           | 2025-10-15        | 3              
Idul            | 88           | 2025-10-01        | 3              
